# FS20C7 - Recommendations in Retail
#### Documentation
----
Authors: Simon Luder, Roman Studer

## Management Summary
<span style="color:red">Maximal eine Seite lang!</span>

### Initial Situation
The scientific report aims to present the essential aspects of a recommender system down to the mathematical basics. The individual steps from the creation of an interaction matrix, similarity matrix to the prediction of a rating for items that a customer has not yet purchased are included. The basis is a dataset which contains transactions of customers. For each transaction information of User Id, Product Id, Order Id, Department of the product, Aisle of the product and if the product has been ordered before are available.

### Procedure
A recommender can be created in several ways. There are two basic procedures. Model-based and memory-based. This document focuses on memory based recommenders which predicts a rating for products using an interaction and similarity matrix. Memory based recommenders can be further divided into two categories. Item based collaborative filtering (IBCF) and user based collaborative filtering (UBCF). The procedure is the same for both. With the help of the list of transactions an interaction matrix is created which records the interactions (in our case orders) for each item for each user. On this basis, a similarity matrix is created which records the similarity of two products in a numerical value. With the interaction matrix and the similarity matrix a prediction matrix can be created, which has basically the same structure as an interaction matrix, but predicts a rating for each user for each product not yet purchased. Based on this prediction, products can then be recommended. Using various metrics, mathematical and heuristic, a recommender is then evaluated. Different methods for calculating the individual matrices can be applied. A comparison is given at the end of the report or under Result.

### Result
<span style="color:blue">What are our deliverables?</span>

## 1 Introduction
<span style="color:blue">Quesitons to answer: Why was the study undertaken? What was the research question, the tested hypothesis or the purpose of the research?</span>

- Initial Situation 
- Why do we build a plug'n'play recommdender?
- Problem description (Aufgabenstellung)

<span style="color:red">Task definition: Official Task definition out of DS-Spaces (in Introduction) + Aufbau des Berichts in die Einführung einbinden.</span>

> The following delivery objects are agreed with the client:
>
> - Written concept for the planned procedure, in particular cleaning up the product shelf and defining the success criteria
> - Documentation (possibly presentation) of the implemented recommender system with explanation of the tested algorithms, the methodical approach and the gained insights (incl. Q&A)
> - Comparison and assessment of the model quality of different model- and memory-based recommender systems with (a) methods of statistical off-line evaluation and (b) assessment of the personalisation of proposals
> - Documentation of the methods, analyses and results of steps 2 and 3 in the form of a scientific report. 
> - Reflection of the scientific procedure by means of a criteria sheet (basis delivery objects 1-4)
>
> In a first step, the assignment is to implement a simple model- or memory-based recommender system itself "from scratch", i.e. without using packages, and to evaluate the model quality "off-line". This should create a small experimental field in which different algorithms can be systematically tested and compared. The knowledge gained about the functionality and programming techniques should be used to make a well-founded selection from solutions offered in packages (see point 2 above). 
>
> In a further step, packages will be used to make further systematic comparisons of several recommender system approaches and then to quantify and visualise the degree of personalisation of the sales recommendations made (see point 3 above). 
>
> The findings from the Challenge are to be documented in a systematic form as a scientific report (point 4 above), which is generated as a PDF from the Code Notebook. Furthermore, the scientific procedure is to be reflected on the basis of a criteria sheet (point 5 above). 

## 2 Theoretical Background 

<span style="color:red">How does a recommender work? What is needed? Explain a recommender system.</span>

- Where do we use recommenders, what kind of problems do they solve?
- What are the different main approaches for recommendersystems (collaborative, content based)
- What is a basic structure of a Recommender
- (Similarity, Interaction)

A recommender is used to suggest products that a user has not yet interacted with, based on their personal preferences. The recommender learns these preferences by orienting himself on the previous interactions of the user and with the help of which he creates a user profile (Model based recommender) or searches for similar users and items (Memory based recommender). Recommenders are best known in online shops. Amazon shows products that 'may interest you' or Netflix suggests movies that you 'might like' based on your personal interests. These systems are of great importance to help customers find what they are looking for as quickly as possible. With a huge product shelf like Amazon has, a customer is dependent on it because it is impossible for them to find products they are not familiar with. There are two distinctions here. Firstly, a recommender can help the customer to discover new things and show them new, unknown preferences. The recommender thus helps the customer to 'explore'. Second, a recommender can also 'exploit' the known preferences of users to recommend products that match the known preferences.  

### 2.1 Main goal
<span style="color:blue">Explain what we want to build. A modular recommender to explore diffrent variants and combinations of recommder systems.</span>
The aim of this report is to compare different compilations of recommenders with each other and to get a better understanding of the structure of a recommender and the impact of different methods on its usability. For this reason, we build a modular recommender from scratch, which includes several methods to create a rating, several methods to calculate a similarity and several methods to create predictions and evaluate the recommender. This allows us to get several different recommendations from the same dataset input. 

### 2.2 Recommender System Algorithms
<span style="color:blue">This section deals with the different approaches for recommender systems. E.g collaborative filtering, content based, hybrid approach.
</span>

#### 2.2.1 User and Item based collaborative filtering

<img src='./images/flow_ibcf.png'>

Collaborative filtering uses different methods to calculate the similarity between two products or two users. In an item-based approach, a product is compared with another by comparing it to another product. The more similar the interactions of customers between these two products are, the more they fit together. 
With the user based approach the same happens, but instead of products, customers are compared with each other. 

A recommender using collaborative filtering needs a certain number of interactions per customer to deliver useful results. Therefore it makes sense to offer generic recommendations to users who have not yet often interacted with the products. Since the preferences of the customer cannot yet be determined exactly. Here a list of "Top Products" or "Most Purchased" is useful. In the **reduction** phase it makes sense to filter out customers with less than n orders and to make generic recommendations to them. The customers with more than n orders are suitable for further processing. The transactions of these customers can be recorded in an **interaction matrix** $I$ with the dimension $user \times item$. The value for $I_{u,i}$ represents the rating of the customer $u$ for item $i$. This rating can be recorded in various forms: 

- A rating can be a simple unary rating which only states if the user $u$ has interacted with the item $i$ at some point. This binary rating can also be applied if the user provides a binary rating. A like button, for example, provides the same unary/binary rating if a transaction is recorded not by buying something but by liking a movie, an image or something similar. 

$$\displaystyle{ r(u,i) =
  \begin{cases}
    1 & \quad \text{if } u \text{ interacted with item } i\\
    0 & \quad \text{otherwise}\\
  \end{cases}}$$

- Another rating counts the number of interactions per user $u$ and item $i$. This provides an ordinal rating. The rating could for example record the number of orders for the same product by the same user:

$$r(u,i) = \sum_{i = 1}^n 1 \text{  , for n of interactions}$$

- If the transaction records a rating that the customer provided himself, like a rating on a scale from 1 to 10. This provided rating can be used to fill $I_{u,i}$ without any calculation. Something to keep in mind using this approach is that a rating 0 does not equal "no interaction" but represents a rating of 0 or "no interest":

$$\displaystyle{ r(u,i) =
  \begin{cases}
    r & \quad \text{if } u \text{ provided a rating for } i\\
    \text{NaN} & \quad \text{otherwise}\\
  \end{cases}}$$

<span style="color:red">Add own rating here</span>

The next step is to use the interaction matrix $I$ and a similarity function, like "cosine similarity", to calculate the similarity between each item or each user. The resulting **Similarity Matrix** $S$ therefore has the dimensions $user \times user$ or $item \times item$. An item-item matrix defines the collaborative filter as item-based collaborative filtering (IBCF) and a user-user matrix defines it as user-based collaborative filtering (UBCF) because the first is looking for similar items and the second for similar users. Not every similarity function is applicable to a UBCF _and_ an IBCF. For example, Pearson correlation is best suited for user-users, i.e. a UBCF, where Cosine similarity gives the best results for item-item, i.e. an IBCF. For example, if we work with an IBCF, we can use the similarity function cosine similarity to determine the similarity $s$ of two items. Cosine similarity works with the intermediate angle function and thus calculates the similarity $s$ vectorized. Since the resulting matrix $S$ is symmetric only the upper or lower triangular matrix has to be calculated: (Three of several methods are described in detail in section 3.4.) 

$$
s(i,j) = \frac{r_{i}\cdot r_{j}}{\|r_{i}\|_{2}\cdot\|r_{j}\|_{2}}
$$

In a next step the similarity matrix can be used to predict ratings for items with which a user has not yet interacted using a prediction function. This prediction matrix $P$ has the same dimensions as the interaction matrix $I$, i. e. $user \times item$. Since the prediction function predicts a rating for each product, this matrix $P$ is necessarily a dense matrix. The information from the prediction matrix can then be used to suggest new items to a user. It is also possible to recommend items directly from the similarity matrix $S$. A KNN algorithm searches for the items that best match each other. Please note that it does not make sense to suggest items that are as similar as possible for every system, as this reduces the diversity of suggestions.

It would now be possible to use the matrices $I, S$ and $P$ to suggest items to a user. To check the effectiveness of the recommender, metrics can be used for evaluation. Numerical metrics include Root Mean Squared Error (RMSE), Mean Absolute Error (MAE) or Normalized Mean Absolute Error (NMAE). These methods check the prediction accuracy by masking items of a user. The actual rating of the user is then compared with the proposed rating of the prediction matrix $P$. Another way to evaluate a recommender is to check the recommender accuracy by comparing the top suggested products with the user's interests. 

#### 2.2.2 Content based filtering

<img src='./images/flow_content.png'>

#### 2.2.3 Hybrid recommender system

### 2.3 Evaluation options
<span style="color:blue">Because we want to compare the efficiency of different recommender systems , we need methods to compare how well they perform, regarding our task. What are suitable methods to evaluate a recommender?</span>

## 3 Methods
<span style="color:blue">Quesitons to answer: When, where, and **how** was the study done? What materials were used or who was included in the study groups (patients, etc.)?</span>

### 3.1 Data reduction
<span style="color:blue">According to our task definition, up to 80 percent if all products can be discarded. Here written are the different methodes used to achieve this task.
</span>

### 3.2 Ratings 

### 3.3 Interaction matrix
<span style="color:blue">The reduced data now needs to be brought into the shape of a interaction matrix. Here documentet is the evaluation and implementation of the different matrix types.
</span>

An interaction matrix lists every interaction between users $u_i$ and items $i_i$. For this reason, the matrix $I$ has the dimensions $u \times i$ . 

### 3.4 Similarity matrix
<span style="color:blue">The similarity is the base of collaborative filtering models. Which similarity measures can be used for this task and what are the differences it these?
</span>

#### 3.4.1 Cosine Similarity
Cosine similarity measures the similarity between two vectors. To achieve this it calculates the angle between the two vectors. With an intermediate angle of $0°$ (the two vectors point in the same direction) the similarity is $1$, ($cos(0) = 1$). For any other angle the value is less than one. In our example we consider products (columns of the interaction matrix) or users (rows of the interaction matrix) as vectors. In our formula are $i$ and $j$ items which each have a vector consisting of all their ratings $r$. Note: This function can be used for UBCF and IBCF.

$$
s(i,j) = \frac{r_{i}\cdot r_{j}}{\|r_{i}\|_{2}\cdot\|r_{j}\|_{2}}
$$

#### 3.4.2 Pearson Correlation
Person Correlation measures the statistical correlation between the ratings of two user's (or items). The formula takes as input two users $u$ and $v$ and calculates a similarity $s$ using the ratings $r$ for every product the two users have in common. It is possible to use pearson correlation to measure the similarity between items as well. But it is said that it yields **worse results** as a cosine similarity for example. The pearson correlation also suffers from problems if there are just a few ratings to compare. A threshold of a number of ratings per user could counteract this problem.

$$
s(u,v) = \frac{\sum_{i\in I_{u}\cap I_{v}}(r_{u,i}-\bar r_{u})(r_{v,i}-\bar r_{v})}{\sqrt{\sum_{i\in I_{u}\cap I_{v}}(r_{u,i}-\bar r_{u})^2} \sqrt{\sum_{i\in I_{u}\cap I_{v}}(r_{v,i}-\bar r_{v})^2}}
$$

#### 3.4.3 Jaccard Similarity
The Jaccard Similarity measures the similarity between two sets $A, B$. The similarity is calculated by dividing the number of elements in the intersection ($\vert{A\cap  B\vert}$) with the size of the union of the two sets ($\vert{A\cup  B\vert}$). The maximum value (total overlap) is $1$. If two sets have no intersection, the value is $0$.

The function looks as follows:

$$
s(A,B) = \frac{\vert{A\cap  B\vert}}{\vert{A\cup  B\vert}}
$$

### 3.5 Implementation
<span style="color:blue">Description of our code structure. We want to implement our recommender in a modular way so that it is possible to easily swap different methods and approaches for an efficient comparison.</span>

## 4 Results
----
<span style="color:blue">This section will summarize the results of the tested algorithms and methods (including metrics like similarity measures and evaluation methods) and explain them per algorithm. It should also include a description of the comparisons.</span>

### 4.1 Metrics per algorithm
<span style="color:blue">Here the results of the methods used are recorded per algorithm. (if possible in a clear, tabular form)</span>

<span style="color:blue">
Structure:
- Model (IBCF, UBCF, SVD etc.)
- Way of Product Reduction
- Used Similarity Measure (Cosine, Pearson, etc.) 
- Results of evaluation Method. (MAE, NMAE, RSME, Manual)
- Despription (maybe with runtime)</span>

### 4.2 Comparison of the results from diffrent combinations of ratings, similarity functions and prediction functions.

<span style="color:blue">This section brings together all previously recorded results. This allows you to compare them and classify them by effectiveness etc.</span>


<span style="color:blue">After the presentation of the results, a comprehensive description of the results is also included. Here the "winner" or best working combination is announced..</span>

## 5 Discussion
----
<span style="color:blue">Quesitons to answer: What might the answer imply and why does it matter? How does it fit in with what other researchers have found? What are the perspectives for future research?</span>


### 5.1 Description of the results

<span style="color:blue">Here the results of the project work are discussed and evaluated.</span>

<span style="color:blue">How meaningful are our results? This section is dedicated to this question and also partly answers the question whether our system is suitable for further use.</span>


<span style="color:blue">The encountered problems (for expample performance issues) should be documented in this section.</span>

## 6 Conclusion and Outlook

<span style='color:red'>Start with short summary and then link to other, similar projects</span>

<span style="color:blue">A review of the project and a short summary of the results.</span>

<span style="color:blue">There are several projects/libraries that contain a similar project (Modular Recommender). It is interesting to make a comparison here. Example: Surprise Recommender library</span>

<span style="color:blue">What information can we take with us? Is it possible to continue the work and build on it?</span>

## 7 Sources
----

## 8 Glossary
----